In [ ]:
#The following notebook contains part of our pipeline 
#to preprocess tiles with a problem specific data-augmentation,
#to train a VGG16 CNN with Random-Rotate-Zoom datagenerator
#and to visualize testing results in form of Guided GradCAMs

%reload_ext autoreload
%autoreload 2
%matplotlib notebook
import sys 
import keras
from keras.models import Sequential, Model
from keras import optimizers
from keras import backend
from keras import applications
from keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D, Dense, Input, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, AveragePooling2D, Concatenate
from image.image_data_generator import ImageDataGenerator
from keras.preprocessing.image import array_to_img, img_to_array
from image.directory_iterator import DirectoryIterator
from image.dataframe_iterator import DataFrameIterator
import numpy as np
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
import pandas as pd
import imgaug as ia
from imgaug import augmenters as iaa
from keras.applications.xception import preprocess_input
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint, Callback, LambdaCallback, CSVLogger
from keras.metrics import categorical_accuracy
from pathlib import Path
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc

In [ ]:
#helper functions
from helper import as_keras_metric, looking_at_augmentation, plot_history
#preprocessing
sys.path.append#("PATH to helper function")



#VGG-16 with BatchNorm + 2. Denselayer
#VGG_base_nottrainable: only toplayer is trainable
#VGG_base_trainable: whole model trainable

auc_roc = as_keras_metric(tf.metrics.auc)

def get_model_classif_VGG_base_nottrainable():
    base_model_VGG = applications.VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    print('Model loaded.')
    
    for layer in base_model_VGG.layers:
        layer.trainable = False
        print("trainable:", layer.name)
    
    x = base_model_VGG.output
    out = GlobalMaxPooling2D(name='1')(x)
    x = BatchNormalization(name='5')(out)
    x = Dropout(0.5, name='6')(x)
    x = Dense(512, activation='relu',name='7') (x)
    x = BatchNormalization(name='8')(x)
    x = Dropout(0.2, name='9')(x)
    x = Dense(256, activation='relu',name='10') (x)
    x = BatchNormalization(name='11')(x)
    
    predictions = (Dense(2, activation='softmax'))(x)
    model_VGG = Model(inputs=base_model_VGG.input, outputs=predictions)

    adam = optimizers.Adam()
    model_VGG.compile(loss='categorical_crossentropy',
                  optimizer=adam,
                  metrics=[categorical_accuracy])

    model_VGG.summary()
    return model_VGG

def get_model_classif_VGG_base_trainable():
    base_model_VGG = applications.VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    print('Model loaded.')
    
    for layer in base_model_VGG.layers:
        layer.trainable = True
        print("trainable:", layer.name)
    
    x = base_model_VGG.output
    out = GlobalMaxPooling2D(name='1')(x)
    x = BatchNormalization(name='5')(out)
    x = Dropout(0.5, name='6')(x)
    x = Dense(512, activation='relu',name='7') (x)
    x = BatchNormalization(name='8')(x)
    x = Dropout(0.5, name='9')(x)
    x = Dense(256, activation='relu',name='10') (x)
    x = BatchNormalization(name='11')(x)
    
    predictions = (Dense(2, activation='softmax'))(x)
    model_VGG = Model(inputs=base_model_VGG.input, outputs=predictions)

    adam = optimizers.Adam()
    model_VGG.compile(loss='categorical_crossentropy',
                  optimizer=adam,
                  metrics=[categorical_accuracy, auc_roc])

    model_VGG.summary()
    return model_VGG

In [ ]:
IMAGE_SIZE = 100 #start with 100x100 upscale in 50x50 steps (150, 200, 250, 300)
IMAGE_CHANNELS = 3
train_batch_size_1 = 128 
val_batch_size = 64

model_name = 'VGG_16'

base_path = #path to base_dir / path to create new dir
if not os.path.exists(base_path):
    os.makedirs(base_path)

train_path = base_path + #insert path to training images
val_path = base_path + #insert path to validation images
test_path = base_path + #insert path to test images

In [ ]:
#Initialize preprocessing and data-generators with included Random-Rotate-Zoom function
train_datagen_aug = ImageDataGenerator(
                rescale=1./255,
                preprocessing_function=seq.augment_images
                )

validation_datagen_aug = ImageDataGenerator(
                rescale=1./255
                )



train_gen = DirectoryIterator(train_path, train_datagen_aug,
                              target_size=(IMAGE_SIZE, IMAGE_SIZE),
                              crop_size = (100, 100),
                              batch_size=train_batch_size_1,
                              class_mode='categorical')

val_gen = DirectoryIterator(val_path,
                            validation_datagen_aug,
                            target_size=(IMAGE_SIZE,IMAGE_SIZE),
                            crop_size = (100, 100),
                            batch_size=val_batch_size,
                            class_mode='categorical')


num_train_samples = train_gen.samples
num_val_samples = val_gen.samples
print(num_train_samples)
print(num_val_samples)

In [ ]:
#helper function to visualize a 9x9 plot 
#of different training images with our dataaugmentation and Random-Rotate-Zoom
looking_at_augmentation(train_gen, path = train_path, batchsize=9)

In [ ]:
#initialize our defined VGG16
model_VGG = get_model_classif_VGG_base_nottrainable()

In [ ]:
from keras_lr_finder import LRFinder
sys.path.append(r"C:\Users\eg38emed\FCD\FCD vs TSC\keras_utils")

#initialize LR-Finder
lr_gen = DirectoryIterator(train_path,
                            train_datagen_aug,
                            target_size = (IMAGE_SIZE,IMAGE_SIZE),
                            crop_size = (100, 100),
                            batch_size = 10000,
                            class_mode = 'categorical')

x_lr_train,y_lr_train = lr_gen.next()
print(x_lr_train.shape)
lr_finder = LRFinder(model_VGG)

In [ ]:
lr_finder.find(x_lr_train, y_lr_train, 0.000001, 0.1, batch_size=64, epochs=4)
lr_finder.plot_loss(n_skip_beginning=1, n_skip_end=1)

In [ ]:
import clr_callback
def get_callbacks_clr(name_weights):
    ReduceLR = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5, verbose=1, mode='auto', cooldown=5, min_lr=0.00005)
    earlystopping = EarlyStopping(monitor='val_categorical_accuracy', min_delta=0.001, patience=10, verbose=1, mode='auto')
    checkpoint = ModelCheckpoint(name_weights, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
    clr = clr_callback.CyclicLR(base_lr=0.0001, max_lr=0.01,
                        step_size=num_train_samples/train_batch_size_1/2)
    csv_logger = CSVLogger(filename = base_path + "\\model_history_{}_toplayer.csv".format(model_name), append=True)
    return [clr, earlystopping, ReduceLR, checkpoint, csv_logger]

name_weights = (base_path + "\\{}_toplayer.h5".format(model_name))
callbacks_list = get_callbacks_clr(name_weights = name_weights)

In [ ]:
history = model_VGG.fit_generator(generator = train_gen, 
                                  steps_per_epoch=len(train_gen), 
                                  validation_data=val_gen,
                                  validation_steps=len(val_gen),
                                  epochs=40, verbose=1,
                                  use_multiprocessing = True, workers=16,
                                  callbacks=callbacks_list)

In [ ]:
#plotting and saving training for VGG_16_basenottrainable
plot_history(history, modelname = model_name, path=base_path)

In [ ]:
#initialize our defined VGG16
model_VGG = get_model_classif_VGG_base_trainable()

In [ ]:
from keras_lr_finder import LRFinder
sys.path.append(r"C:\Users\eg38emed\FCD\FCD vs TSC\keras_utils")

#initialize LR-Finder
lr_gen = DirectoryIterator(train_path,
                            train_datagen_aug,
                            target_size = (IMAGE_SIZE,IMAGE_SIZE),
                            crop_size = (100, 100),
                            batch_size = 10000,
                            class_mode = 'categorical')

x_lr_train,y_lr_train = lr_gen.next()
print(x_lr_train.shape)
lr_finder = LRFinder(model_VGG)

In [ ]:
lr_finder.find(x_lr_train, y_lr_train, 0.000001, 0.1, batch_size=64, epochs=4)
lr_finder.plot_loss(n_skip_beginning=1, n_skip_end=1)

In [ ]:
import clr_callback
def get_callbacks_clr(name_weights):
    ReduceLR = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5, verbose=1, mode='auto', cooldown=5, min_lr=0.00005)
    earlystopping = EarlyStopping(monitor='val_categorical_accuracy', min_delta=0.001, patience=10, verbose=1, mode='auto')
    checkpoint = ModelCheckpoint(name_weights, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
    clr = clr_callback.CyclicLR(base_lr=0.0001, max_lr=0.01,
                        step_size=num_train_samples/train_batch_size_1/2)
    csv_logger = CSVLogger(filename = base_path + "\\model_history_{}_all.csv".format(model_name), append=True)
    return [clr, earlystopping, ReduceLR, checkpoint, csv_logger]

name_weights = (base_path + "\\{}_all.h5".format(model_name))
callbacks_list = get_callbacks_clr(name_weights = name_weights)

In [ ]:
history = model_VGG.fit_generator(generator = train_gen, 
                                  steps_per_epoch=len(train_gen), 
                                  validation_data=val_gen,
                                  validation_steps=len(val_gen),
                                  epochs=40, verbose=1,
                                  use_multiprocessing = True, workers=16,
                                  callbacks=callbacks_list)

In [ ]:
#plotting and saving training for VGG_16_basenottrainable
plot_history(history, modelname = model_name, path=base_path)

In [ ]:
#helper functions for visualization
def target_category_loss(x, category_index, nb_classes):
    return tf.multiply(x, K.one_hot([category_index], nb_classes))

def target_category_loss_output_shape(input_shape):
    return input_shape

def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)

def load_image(path):
    img = image.load_img(path, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    x = image.img_to_array(img)
    x = (x - np.min(x))/np.ptp(x)
    x = np.expand_dims(x, axis=0)

    return x

#registering guidedrelu as a tensorflow gradient
#the modified gradient only backpropagates gradients which had a positive activation during forward pass
def register_gradient():
    if "GuidedBackProp" not in ops._gradient_registry._registry:
        @ops.RegisterGradient("GuidedBackProp")
        def _GuidedBackProp(op, grad):
            dtype = op.inputs[0].dtype
            return grad * tf.cast(grad > 0., dtype) * \
                tf.cast(op.inputs[0] > 0., dtype)


#Generating the Saliency Map of the Image with respect to the prediction & the given model        
def compile_saliency_function(model, activation_layer='block5_conv3'):
    input_img = model.input
    layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])
    layer_output = layer_dict[activation_layer].output
    max_output = K.max(layer_output, axis=3)
    #Keras function returning the saliency map given an image input
    saliency = K.gradients(K.sum(max_output), input_img)[0]
    return K.function([input_img, K.learning_phase()], [saliency])

#to implement guided backpropagation we need to duplicate the model with modified gradients & linear activation
#replacing the model activations with linear activations
#linear activation ensures that when computing derivatives of a class output, features which minimize the otehr classes arent encluded
def modify_backprop(model, name, name_weights):
    g = tf.get_default_graph()
    with g.gradient_override_map({'Relu': name}):

        # get layers that have an activation
        layer_dict = [layer for layer in model.layers[1:]
                      if hasattr(layer, 'activation')]

        # replace relu activation
        for layer in layer_dict:
            if layer.activation == keras.activations.relu:
                layer.activation = tf.nn.relu

        # re-instanciate a new model
        new_model = get_model_classif_VGG_base_trainable()
        new_model.load_weights(name_weights)
    return new_model

def deprocess_image(x):
    '''
    Same normalization as in:
    https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
    '''
    if np.ndim(x) > 3:
        x = np.squeeze(x)
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_dim_ordering() == 'th':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
def grad_cam(input_model, image, category_index, layer_name):
    #defining the layer & the position of our class on the prediction vector
    nb_classes = 2
    target_layer = lambda x: target_category_loss(x, category_index, nb_classes)
    x = input_model.layers[-1].output
    x = Lambda(target_layer, output_shape=target_category_loss_output_shape)(x)
    model = keras.models.Model(input_model.layers[0].input, x)
    
    #getting the output of the layers with the predefined layername
    loss = K.sum(model.layers[-1].output)
    conv_output = [l for l in model.layers if l.name is layer_name][0].output
    
    #Gradient for our class with regard to the output-feature map of the predefined layer
    grads = normalize(K.gradients(loss, conv_output)[0])
    #Keras function taking image input and accessing the quantities of:
    #"grads" (mean intensity gradient for a specific class) & 
    #"conv_output" (output featuremaps)
    gradient_function = K.function([model.layers[0].input], [conv_output, grads])
    output, grads_val = gradient_function([image])
    #values of grads & conv_output as numpy arrays
    output, grads_val = output[0, :], grads_val[0, :, :, :]
    #normalising gradients via GlobalPooling
    weights = np.mean(grads_val, axis = (0, 1))
    #initializing array with right shape
    cam = np.ones(output.shape[0 : 2], dtype = np.float32)
    #multiplying each channel in the feature-map by "how important it is" with respect to the given class
    for i, w in enumerate(weights):
        cam += w * output[:, :, i]
    
    #formating for visualisation purposes: image, heatmap, GRAD CAM
    cam = cv2.resize(cam, (IMAGE_SIZE, IMAGE_SIZE))
    #sudo Relu
    cam = np.maximum(cam, 0)
    #Normalizing for Visualisation
    heatmap = cam / np.max(cam)
    heatmap = heatmap * 255
    
    
    image = image[0, :]
    image -= np.min(image)
    image = np.minimum(image, 255)
    image = 255 * image / np.max(image)

    cam = cv2.applyColorMap(np.uint8(heatmap), cv2.COLORMAP_JET)
    #cam[np.where(cam < 95)] = 0
    #superimposed_img = cv2.addWeighted(image, 0.7, cam, 0.3, 0)
    cam = np.float32(cam) + np.float32(image)
    superimposed_img = 255 * cam / np.max(cam)
    #print(np.uint8(cam))
    return np.uint8(superimposed_img), heatmap

def guided_grad_cam(model, preprocessed_input, heatmap, name_weights):
    register_gradient()
    guided_model = modify_backprop(model, 'GuidedBackProp', name_weights=name_weights)
    saliency_fn = compile_saliency_function(guided_model)
    saliency = saliency_fn([preprocessed_input, 0])
    gradcam = saliency[0] * heatmap[..., np.newaxis]
    ggc= deprocess_image(gradcam)
    return ggc

In [ ]:
batch_size_gradcam= 1
datagen_gradcam = ImageDataGenerator(rescale=1./255,
                                  rotate_random_zoom_crop=True)

gradcam_generator = datagen_gradcam.flow_from_directory(test_path, 
                                                  batch_size = batch_size_gradcam, 
                                                  shuffle=False,
                                                  #target_size=(IMAGE_SIZE, IMAGE_SIZE), 
                                                  crop_size = (100,100),
                                                  class_mode='categorical')

In [ ]:
# Setup the data_frame: Path, ID, Label, Image
print(test_path)

fcd = test_path +  '/a_fcd'
tsc = test_path + '/b_tsc'

#FCD folder
df = pd.DataFrame({'path': glob(os.path.join(fcd,'*.png'))})
df['id'] = df.path.map(lambda x: x.split('/')[7].split('.')[0]) 
df['label'] = 0

#TSC folder
df_pos = pd.DataFrame({'path': glob(os.path.join(tsc,'*.png'))})
df_pos['id'] = df_pos.path.map(lambda x: x.split('/')[7].split('.')[0]) #Server id split: 7 
df_pos['label'] = 1

df_train = pd.concat([df, df_pos])

print(len(df_train))
print(df_train.head())

In [ ]:
gradcam_generator.reset()
predictions = model.predict_generator(gradcam_generator, 
                                      steps=len(gradcam_generator), 
                                      workers=8, 
                                      use_multiprocessing=True, 
                                      verbose=1)

In [ ]:
# Get the true labels
y = to_categorical(df_train['label'])
y_label = np.argmax(y, axis=1)
print(y_label)

df_preds = pd.DataFrame(predictions, columns=['FCD', 'TSC'])
print(df_preds.head())

# Get the predicted labels as probabilities
y_pred = df_preds['TSC']
y_predictions = np.asarray(y_pred.round(3))
print(y_predictions)

#rounding predictions + make them int --> compare to label / heatmap
y_pred = y_pred.round()
y_pred_int = []
for i in y_pred:
    if i == 1.0:
        y_pred_int.append(1)
    else:
        y_pred_int.append(0)

y_pred_int = np.asarray(y_pred_int)
print((y_pred_int.shape))

In [ ]:
#view AUCROC and ConfusionMatrix to Quality-Check
plot_roc(y_label, y_pred, modelname='try', path = base_path)

In [ ]:
cm = confusion_matrix(y_label, y_pred.round())
cm_plot_labels = ['FCD', 'TSC']

plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix VGG')

In [ ]:
filenames = gradcam_generator.filenames
incorrect = np.where(y_pred!=y_label)[0]
for i, incorrect in enumerate(incorrect):
    print(incorrect)
    print(filenames[incorrect])

In [ ]:
filenames = gradcam_generator.filenames
print(filenames)

preprocessed_input = load_image(test_path + 'b_tsc/N1714-17 II2 (256-17) - 2018-12-07 11.41.42_TSC974.png')

predictions = model.predict(preprocessed_input)
predicted_class = np.argmax(predictions)
print(predictions)
print(predicted_class)

%matplotlib notebook
cam, heatmap = grad_cam(model, preprocessed_input, predicted_class, "block5_conv3")

plt.imshow(cv2.cvtColor(cam, cv2.COLOR_BGR2RGB))
#plt.imshow(np.uint8(heatmap*255), cmap='viridis')
#cv2.imwrite("gradcam.jpg", cam)
#cv2.imwrite("heatmap.jpg", heatmap)

%matplotlib notebook
ggc =  guided_grad_cam(model, preprocessed_input, heatmap, name_weights=name_weights)   
plt.imshow(cv2.cvtColor(ggc, cv2.COLOR_BGR2RGB), cmap= 'jet')
#plt.imshow(ggc)

In [ ]:
#plot most confident FCD IIb Heatmaps + GradCAMs and save in new subdir
mostconfident = np.where((y_label==0) & (y_label==y_pred_int))[0]
print(len(mostconfident))
model_name = 'FCD'

GRAD_CAM_PATH_FCD = (base_path + '/{}/'.format(model_name))
if not os.path.exists(GRAD_CAM_PATH_FCD):
    os.makedirs(GRAD_CAM_PATH_FCD)

fig, ax = plt.subplots(4, 4, figsize=(12,12), squeeze=False)
fig.subplots_adjust(hspace=0.5, wspace=0.5)
plt.suptitle('GRAD CAM: {}'.format(name_weights), fontsize=16)

In [ ]:
k=0
j=4

for n in range(len(mostconfident)):
    print('Getting GRAD-CAMs for FCD {} - {}'.format(k,j))
    for i , correct in enumerate(mostconfident[k:j]):
        
        preprocessed_input, y  = next(gradcam_generator)
        im = (preprocessed_input - np.min(preprocessed_input))/np.ptp(preprocessed_input)
        im = np.squeeze(im, axis=0)
        print(im.shape)
        predicted_class = np.argmax(y)
        cam, heatmap = grad_cam(model, preprocessed_input, predicted_class, "block5_conv3")
        ggc = guided_grad_cam(model, preprocessed_input, heatmap, name_weights=name_weights)
        ax[0,i].set_title(filenames[correct], fontsize=6)
        ax[0,i].imshow(im)

        ax[1,i].set_title("Predicted: {}, Label: {}".format(y_predictions[correct], y_label[correct]), fontsize=8)
        map = ax[1,i].imshow(heatmap, interpolation="none", cmap = 'rainbow')

        ax[2,i].imshow(cv2.cvtColor(cam, cv2.COLOR_BGR2RGB))
        ax[3,i].imshow(cv2.cvtColor(ggc, cv2.COLOR_BGR2RGB))

        ax[0,0].set_ylabel('Original')
        ax[1,0].set_ylabel('Heatmap')
        ax[2,0].set_ylabel('GRAD CAM')
        ax[3,0].set_ylabel('Guided GRAD CAM')
    k+=8
    j+=8
    plt.show()
    fig.savefig(GRAD_CAM_PATH_FCD + '/Mostconfident_FCDIIb_CAMs_{}-{}.png' .format(k,j), dpi=100)

In [ ]:
#plot most confident TSC GradCAMs and save in new subdir
model_name = 'TSC'

GRAD_CAM_PATH_TSC = (base_path + '/{}/'.format(model_name))
if not os.path.exists(GRAD_CAM_PATH_TSC):
    os.makedirs(GRAD_CAM_PATH_TSC)

correct_tsc = np.where((y_label==1) & (y_label==y_pred_int))[0]
print("Found {} confident correct TSC labels: {}" .format(len(correct_tsc), correct_tsc))

fig, ax = plt.subplots(4, 4, figsize=(12,12), squeeze=False)
fig.subplots_adjust(hspace=0.5, wspace=0.5)
plt.suptitle('GRAD CAM: {}'.format(name_weights), fontsize=16)

In [ ]:
k=0
j=4

for n in range(len(correct_tsc)):
    print('Getting GRAD-CAMs for TSC {} - {}'.format(k,j))
    for i , correct in enumerate(correct_tsc[k:j]):
        preprocessed_input, y  = next(gradcam_generator)
        im = (preprocessed_input - np.min(preprocessed_input))/np.ptp(preprocessed_input)
        im = np.squeeze(im, axis=0)
        print(im.shape)
        predicted_class = np.argmax(y)
        cam, heatmap = grad_cam(model, preprocessed_input, predicted_class, "block5_conv3")
        ggc = guided_grad_cam(model, preprocessed_input, heatmap, name_weights=name_weights)
        ax[0,i].set_title(filenames[correct], fontsize=6)
        ax[0,i].imshow(im)

        ax[1,i].set_title("Predicted: {}, Label: {}".format(y_predictions[correct], y_label[correct]), fontsize=8)
        map = ax[1,i].imshow(heatmap, interpolation="none", cmap = 'rainbow')

        ax[2,i].imshow(cv2.cvtColor(cam, cv2.COLOR_BGR2RGB))
        ax[3,i].imshow(cv2.cvtColor(ggc, cv2.COLOR_BGR2RGB))

        ax[0,0].set_ylabel('Original')
        ax[1,0].set_ylabel('Heatmap')
        ax[2,0].set_ylabel('GRAD CAM')
        ax[3,0].set_ylabel('Guided GRAD CAM')
    k+=4
    j+=4
    plt.show()
    fig.savefig(GRAD_CAM_PATH_TSC + '/TSC_Mostconfident_CAMs_{}-{}.png' .format(k,j), dpi=100)